In [2]:
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By
from seleniumwire import webdriver
import time
import pandas as pd

In [81]:
profile = profiles.Windows() # or .Android
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
driver = Chrome(profile, options=options,
                uc_driver=False
                )

driver.get("https://store.steampowered.com/search/?filter=topsellers&category1=998&deck_compatibility=2&supportedlang=english&ndl=1")
for i in range(30):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
start = 1
end = 250
time.sleep(3)
show_select = driver.find_element(By.XPATH, '//*[@id="advsearchform"]/div[1]/div/div[1]')


game_links = []
game_prices = {}
game_info = []
dev_info = []
links = []
genre = []
game_tags = []
try:
    games = driver.find_elements(By.XPATH, f'//*[@id="search_resultsRows"]/a[position()>{start} and position()<{end}]')
    if len(games) == 0: raise ValueError 
    for game in games:
        game_link = game.get_attribute('href')
        game_links.append(game_link)
        game_prices[game.find_element(By.XPATH, './div[2]/div[1]/span').text]= game.find_element(By.CLASS_NAME, 'discount_prices').text


    for link in game_links:
        driver.get(link)
        time.sleep(3)
        try:
            # game.find_element(By.XPATH, '//*[@id="app_agegate"]/div[1]/div[2]/div').get_attribute('href')
            day = driver.find_element(By.ID, 'ageDay')
            day.click()
            day_menu = driver.find_element(By.XPATH, "//option[@value='4']")
            day_menu.click()
            month = driver.find_element(By.ID, 'ageMonth')
            month.click()
            month_menu = driver.find_element(By.XPATH, "//option[@value='April']")
            month_menu.click()
            year = driver.find_element(By.ID, 'ageYear')
            year.click()
            year_menu = driver.find_element(By.XPATH, "//option[@value='2000']")
            year_menu.click()
            view_page = driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]/span')
            view_page.click()
            print('Age gate passed, continuing...')
            time.sleep(3)

        except:
            print('No age gate, continuing...')
        driver.execute_script("""
        for(let script of document.querySelectorAll('script')) script.remove()
        """)
        try:
            game_name = driver.find_element(By.XPATH, '//*[@id="appHubAppName"]').text
            dev_info = driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/span')
            tags = driver.find_element(By.XPATH, '//*[@id="category_block"]/div[1]')
            genre = driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/span')
            try:
                game_info.append({
                "Name": game_name,
                "Review": driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/span[1]').text,
                "Number Of Reviews": driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/meta[1]').get_attribute('content'),
                "Genre": genre.text.split('\n'),
                "Tags": tags.text.split('\n'),
                "Dev_Info": dev_info.text.split('\n'),
                "Developer": driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[1]/a').text,
                "Publisher": driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[2]/a').text,
                "Release_Date": driver.find_element(By.XPATH , '//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]').text,
                "link": link,
                "Price": game_prices[game_name]
                })
            except:
                game_info.append({
                "Name": game_name,
                "Review": driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/span[1]').text,
                "Number Of Reviews": driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/meta[1]').get_attribute('content'),
                "Genre": genre.text.split('\n'),
                "Tags": tags.text.split('\n'),
                "Dev_Info": dev_info.text.split('\n'),
                "Developer": driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[1]/a').text,
                "Publisher": driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[2]/a').text,
                "Release_Date": driver.find_element(By.XPATH , '//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]').text,
                "link": link,
                "Price": None
                })
            

    
    
        

            links.append(link)
            # print(game_name)
        except:
            print('Exception occured, continuing...')
except:
    print("Couldn't fetch any more games...")
    df = pd.DataFrame(game_info)
    df.to_csv('steam_games.csv', index=False)
    driver.quit()

df = pd.DataFrame(game_info)
df.to_csv('steam_games.csv', index=False)
print(len(game_links))
print(game_links)
driver.quit()

No age gate, continuing...
Couldn't fetch any more games...
19
['https://store.steampowered.com/app/1096530/Solasta_Crown_of_the_Magister/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/212680/FTL_Faster_Than_Light/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/1850160/Poly_Bridge_3/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/1424330/Wobbledogs/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/1250/Killing_Floor/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/558990/Opus_Magnum/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/1307550/Craftopia/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/2328760/Pinball_FX/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/251060/Wargame_Red_Dragon/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/445220/Avorion/?snr=1_7_7_7000_150_11', 'https://store.steampowered.com/app/304650/SUNLESS_SEA/?snr=1_7_7_7000_150_11', 'https://store.steampo

In [64]:
# Driver for testing...
profile = profiles.Windows() # or .Android
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
driver = Chrome(profile, options=options,
                uc_driver=False
                )

driver.get("https://store.steampowered.com/search/?filter=topsellers&category1=998&deck_compatibility=2&supportedlang=english&ndl=1")

In [70]:
# Cell for testing...
games = driver.find_elements(By.XPATH, f'//*[@id="search_resultsRows"]/a[position()>{2} and position()<{9}]')

game_links = []
game_prices = {}
game_info = []
dev_info = []
links = []
genre = []
game_tags = []
for game in games:
    game_link = game.get_attribute('href')
    game_links.append(game_link)
    game_prices[game.find_element(By.XPATH, './div[2]/div[1]/span').text]= game.find_element(By.CLASS_NAME, 'discount_prices').text


print(game_prices['Lethal Company'])
    

$5.79
